# Apparel images dataset Classification 
### Transfer Learning using keras application

Transfer learning is a machine learning method where a model developed for a task is reused as the starting point for a model on a second task

<img src="https://learnopencv.com/wp-content/uploads/2019/05/transfer-learning-1024x574.jpg" alt="drawing" width="500"/>

In this notebook the model use resnet 50 model and add some final layer which are going to be trained.

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Models 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPooling2D

import os

folder_path = '../DATASET/CNN_DATASET_PAKAIAN'
for dirname, _, filenames in os.walk(folder_path):
    print(dirname)

../DATASET/CNN_DATASET_PAKAIAN
../DATASET/CNN_DATASET_PAKAIAN\black_dress
../DATASET/CNN_DATASET_PAKAIAN\black_pants
../DATASET/CNN_DATASET_PAKAIAN\black_shirt
../DATASET/CNN_DATASET_PAKAIAN\black_shoes
../DATASET/CNN_DATASET_PAKAIAN\black_shorts
../DATASET/CNN_DATASET_PAKAIAN\blue_dress
../DATASET/CNN_DATASET_PAKAIAN\blue_pants
../DATASET/CNN_DATASET_PAKAIAN\blue_shirt
../DATASET/CNN_DATASET_PAKAIAN\blue_shoes
../DATASET/CNN_DATASET_PAKAIAN\blue_shorts
../DATASET/CNN_DATASET_PAKAIAN\brown_pants
../DATASET/CNN_DATASET_PAKAIAN\brown_shoes
../DATASET/CNN_DATASET_PAKAIAN\brown_shorts
../DATASET/CNN_DATASET_PAKAIAN\green_pants
../DATASET/CNN_DATASET_PAKAIAN\green_shirt
../DATASET/CNN_DATASET_PAKAIAN\green_shoes
../DATASET/CNN_DATASET_PAKAIAN\green_shorts
../DATASET/CNN_DATASET_PAKAIAN\red_dress
../DATASET/CNN_DATASET_PAKAIAN\red_pants
../DATASET/CNN_DATASET_PAKAIAN\red_shoes
../DATASET/CNN_DATASET_PAKAIAN\white_dress
../DATASET/CNN_DATASET_PAKAIAN\white_pants
../DATASET/CNN_DATASET_PAKAIAN

In [ ]:
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
train_datagen = ImageDataGenerator(
                fill_mode = 'nearest',
                validation_split=0.2
)

In [ ]:
train_generator=train_datagen.flow_from_directory(
    folder_path,
    target_size=(255,255),
    color_mode='rgb',
    class_mode='categorical',
    subset='training',
)
validation_generator=train_datagen.flow_from_directory(
    folder_path,
    target_size=(255,255),
    color_mode='rgb',
    class_mode='categorical',
    subset='validation',
)

Found 9115 images belonging to 24 classes.
Found 2270 images belonging to 24 classes.


In [ ]:
tf.keras.applications.imagenet_utils.preprocess_input(
    train_generator, data_format=None, mode='caffe'
)

In [ ]:
from keras.applications import ResNet50
i = tf.keras.layers.Input([None, None, 3], dtype = tf.uint8)
x = tf.cast(i, tf.float32)
x = tf.keras.applications.mobilenet.preprocess_input(x)
core = tf.keras.applications.ResNet50(input_shape=(255,255,3), include_top=False)
x = core(x)
model = tf.keras.Model(inputs=[i], outputs=[x])

image = tf.image.decode_png(tf.io.read_file('file.png'))
result = model(image)

In [ ]:
from keras.applications import ResNet50
model = tf.keras.models.Sequential([
    ResNet50(input_shape=(255,255,3), include_top=False),
])
for layer in model.layers:
  layer.trainable = False

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(24, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 conv2d_3 (Conv2D)           (None, 6, 6, 64)          1179712   
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 3, 3, 64)         0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 576)               0         
                                                                 
 dense_6 (Dense)             (None, 64)                36928     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                      

In [ ]:
model.compile(loss='categorical_crossentropy',
             optimizer=keras.optimizers.Adam(learning_rate=0.01),
             metrics=['accuracy'], run_eagerly=True)

In [ ]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=5,
                    verbose=1,
                    )

Epoch 1/5
 87/285 [========>.....................] - ETA: 47:50 - loss: 5.8804 - accuracy: 0.0837

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

In [ ]:
model.save('/kaggle/working/model.h5')